In [1]:
from binance.spot import Spot
client = Spot(base_url ='https://api2.binance.com')
import pandas as pd
from datetime import datetime as dt
from math import ceil

In [2]:
# set input parameters
# pairing_list = [INPT]
end_date = dt.now()
timeframe_d = 365

In [3]:
# col names
cols = [
    'Open_time',
    'Open',
    'high',
    'low',
    'close',
    'volume',
    'close_time',
    'quote_asset_volume',
    'n_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]

In [32]:
def get_endtimes(range_d, final_datetime):
    calls_n = ceil(range_d*24*60/1000)
    # convert final datetime to unix timestamp
    final_datetime= final_datetime.replace(second=0, microsecond=0)
    final_ts = int(dt.timestamp(final_datetime)*1000)
    # create anchor values
    anchors = range(-calls_n,1)
    # turn anchor values into timestamps for the api call
    call_times = [
        60*1000*1000*anchor + final_ts
        for anchor in anchors
    ]
    return call_times

In [38]:
def klines_call(pairing,end):
    data=client.klines(symbol=pairing,interval='1m',limit=1000,endTime=end)
    df = pd.DataFrame(data=data)
    return df

In [47]:
def get_pairing_data(pairing,range_d,final_datetime='now'):
    if final_datetime == 'now':
        end = dt.now()
    else:
        end = final_datetime
    endtimes_list = get_endtimes(range_d,end)
    df_list = [
        klines_call(pairing,time)
        for time in endtimes_list
    ]
    df = pd.concat(df_list)
    df.columns = cols
    df.drop(['ignore'],axis=1,inplace=True)
    df.loc[:,'pairing'] = pairing
    df.reset_index(inplace=True,drop=True)
    return df

In [ ]:
def get_save_data():
    #df für ein Pairing wird erstellt
    # df wird sinnvoll benannt abgespeichert

In [48]:
df_comp = get_pairing_data('SOLBUSD',5)

In [49]:
df_comp

,Open_time,Open,high,low,close,volume,close_time,quote_asset_volume,n_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,pairing
0,1644402960000,112.89000000,112.96000000,112.85000000,112.85000000,107.41000000,1644403019999,12127.32800000,45,25.48000000,2877.37860000,SOLBUSD
1,1644403020000,112.86000000,112.96000000,112.79000000,112.91000000,90.91000000,1644403079999,10259.40640000,43,34.23000000,3863.41230000,SOLBUSD
2,1644403080000,112.92000000,113.06000000,112.91000000,113.00000000,403.81000000,1644403139999,45629.86270000,47,387.82000000,43823.89270000,SOLBUSD
3,1644403140000,113.01000000,113.11000000,113.00000000,113.07000000,283.70000000,1644403199999,32077.03930000,49,169.53000000,19166.73080000,SOLBUSD
4,1644403200000,113.08000000,113.14000000,113.08000000,113.09000000,109.66000000,1644403259999,12403.41870000,49,103.42000000,11697.75080000,SOLBUSD
...,...,...,...,...,...,...,...,...,...,...,...,...
995,1644942660000,102.62000000,102.66000000,102.48000000,102.48000000,230.19000000,1644942719999,23608.81270000,36,52.20000000,5356.01440000,SOLBUSD
996,1644942720000,102.49000000,102.65000000,102.49000000,102.55000000,45.07000000,1644942779999,4624.24970000,19,8.04000000,825.11460000,SOLBUSD
997,1644942780000,102.53000000,102.64000000,102.53000000,102.61000000,50.03000000,1644942839999,5133.10760000,22,14.35000000,1471.64970000,SOLBUSD
998,1644942840000,102.62000000,102.77000000,102.62000000,102.68000000,256.08000000,1644942899999,26301.20310000,55,206.03000000,21159.73670000,SOLBUSD
